In [1]:
import os
os.environ['KAGGLE_USERNAME'] = "" # username from the json file
os.environ['KAGGLE_KEY'] = "" # key from the json file
!kaggle datasets download -d andrewmvd/ocular-disease-recognition-odir5k

100% 1.62G/1.62G [00:18<00:00, 133MB/s]
100% 1.62G/1.62G [00:18<00:00, 96.0MB/s]


In [2]:
!unzip "ocular-disease-recognition-odir5k.zip" -d "/content"

Streaming output truncated to the last 5000 lines.
  inflating: /content/preprocessed_images/2179_left.jpg  
  inflating: /content/preprocessed_images/2179_right.jpg  
  inflating: /content/preprocessed_images/217_left.jpg  
  inflating: /content/preprocessed_images/217_right.jpg  
  inflating: /content/preprocessed_images/2180_left.jpg  
  inflating: /content/preprocessed_images/2180_right.jpg  
  inflating: /content/preprocessed_images/2181_left.jpg  
  inflating: /content/preprocessed_images/2181_right.jpg  
  inflating: /content/preprocessed_images/2182_left.jpg  
  inflating: /content/preprocessed_images/2182_right.jpg  
  inflating: /content/preprocessed_images/2183_left.jpg  
  inflating: /content/preprocessed_images/2183_right.jpg  
  inflating: /content/preprocessed_images/2184_left.jpg  
  inflating: /content/preprocessed_images/2184_right.jpg  
  inflating: /content/preprocessed_images/2185_left.jpg  
  inflating: /content/preprocessed_images/2185_right.jpg  
  inflating: /c

In [3]:
import numpy as np
import pandas as pd
import cv2
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
df = pd.read_csv("full_df.csv")
df.head(3)

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg


In [5]:
def has_cataract(text):
    if "cataract" in text:
        return 1
    else:
        return 0

In [6]:
df["left_cataract"] = df["Left-Diagnostic Keywords"].apply(lambda x: has_cataract(x))
df["right_cataract"] = df["Right-Diagnostic Keywords"].apply(lambda x: has_cataract(x))

In [7]:
left_cataract = df.loc[(df.C ==1) & (df.left_cataract == 1)]["Left-Fundus"].values
left_cataract[:15]

array(['0_left.jpg', '81_left.jpg', '103_left.jpg', '119_left.jpg',
       '254_left.jpg', '294_left.jpg', '330_left.jpg', '448_left.jpg',
       '465_left.jpg', '477_left.jpg', '553_left.jpg', '560_left.jpg',
       '594_left.jpg', '611_left.jpg', '625_left.jpg'], dtype=object)

In [8]:
right_cataract = df.loc[(df.C ==1) & (df.right_cataract == 1)]["Right-Fundus"].values
right_cataract[:15]

array(['24_right.jpg', '81_right.jpg', '112_right.jpg', '188_right.jpg',
       '218_right.jpg', '345_right.jpg', '354_right.jpg', '477_right.jpg',
       '553_right.jpg', '560_right.jpg', '625_right.jpg', '726_right.jpg',
       '769_right.jpg', '949_right.jpg', '955_right.jpg'], dtype=object)

In [9]:
print("Number of images in left cataract: {}".format(len(left_cataract)))
print("Number of images in right cataract: {}".format(len(right_cataract)))

Number of images in left cataract: 304
Number of images in right cataract: 290


In [10]:
left_normal = df.loc[(df.C ==0) & (df["Left-Diagnostic Keywords"] == "normal fundus")]["Left-Fundus"].sample(250,random_state=42).values
right_normal = df.loc[(df.C ==0) & (df["Right-Diagnostic Keywords"] == "normal fundus")]["Right-Fundus"].sample(250,random_state=42).values
right_normal[:15]

array(['2964_right.jpg', '680_right.jpg', '500_right.jpg',
       '2368_right.jpg', '2820_right.jpg', '2769_right.jpg',
       '2696_right.jpg', '2890_right.jpg', '940_right.jpg',
       '2553_right.jpg', '3371_right.jpg', '3042_right.jpg',
       '919_right.jpg', '3427_right.jpg', '379_right.jpg'], dtype=object)

In [11]:
cataract = np.concatenate((left_cataract,right_cataract),axis=0)
normal = np.concatenate((left_normal,right_normal),axis=0)

In [12]:
print(len(cataract),len(normal))

594 500


In [13]:
from tensorflow.keras.preprocessing.image import load_img,img_to_array
dataset_dir = "preprocessed_images/"
image_size=224
labels = []
dataset = []
def create_dataset(image_category,label):
    for img in tqdm(image_category):
        image_path = os.path.join(dataset_dir,img)
        try:
            image = cv2.imread(image_path,cv2.IMREAD_COLOR)
            image = cv2.resize(image,(image_size,image_size))
        except:
            continue
        
        dataset.append([np.array(image),np.array(label)])
    random.shuffle(dataset)
    return dataset

In [14]:
dataset = create_dataset(cataract,1)

100%|██████████| 594/594 [00:02<00:00, 247.18it/s]


In [15]:
len(dataset)

588

In [16]:
dataset = create_dataset(normal,0)

100%|██████████| 500/500 [00:02<00:00, 239.53it/s]


In [17]:
len(dataset)

1088

In [19]:
x = np.array([i[0] for i in dataset]).reshape(-1,image_size,image_size,3)
y = np.array([i[1] for i in dataset])

In [20]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [21]:
from tensorflow.keras.applications.vgg19 import VGG19
vgg = VGG19(weights="imagenet",include_top = False,input_shape=(image_size,image_size,3))

80142336/80134624 [==============================] - 0s 0us/step


In [22]:
for layer in vgg.layers:
    layer.trainable = False

In [23]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense
model = Sequential()
model.add(vgg)
model.add(Flatten())
model.add(Dense(2,activation="sigmoid"))

In [24]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
checkpoint = ModelCheckpoint("vgg19.h5",monitor="val_acc",verbose=1,save_best_only=True,
                             save_weights_only=False,period=1)
earlystop = EarlyStopping(monitor="val_acc",patience=5,verbose=1)

In [ ]:
history = model.fit(x_train,y_train,batch_size=32,epochs=5,validation_data=(x_test,y_test),
                    verbose=1,callbacks=[checkpoint,earlystop])

Epoch 1/5
28/28 [==============================] - 15s 241ms/step - loss: 1.6058 - accuracy: 0.8287 - val_loss: 0.7971 - val_accuracy: 0.9587
Epoch 2/5
28/28 [==============================] - 5s 169ms/step - loss: 0.8701 - accuracy: 0.9410 - val_loss: 0.2595 - val_accuracy: 0.9679
Epoch 3/5
28/28 [==============================] - 5s 171ms/step - loss: 0.0842 - accuracy: 0.9793 - val_loss: 0.2319 - val_accuracy: 0.9817
Epoch 4/5
28/28 [==============================] - 5s 174ms/step - loss: 0.0742 - accuracy: 0.9856 - val_loss: 0.1232 - val_accuracy: 0.9908
Epoch 5/5
28/28 [==============================] - 5s 176ms/step - loss: 0.0428 - accuracy: 0.9898 - val_loss: 0.0626 - val_accuracy: 0.9862


In [ ]:
loss,accuracy = model.evaluate(x_test,y_test)
print("loss:",loss)
print("Accuracy:",accuracy)

7/7 [==============================] - 1s 144ms/step - loss: 0.0626 - accuracy: 0.9862
loss: 0.06257467716932297
Accuracy: 0.9862385392189026


In [27]:
from keras.preprocessing import image

img_width, img_height = 224, 224
img = image.load_img('/content/ODIR-5K/ODIR-5K/Training Images/0_left.jpg', target_size = (img_width, img_height, 3))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)
np.argmax(model.predict(img), axis=1)

array([1])

In [28]:
model.save("model2.h5")